In [1]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
#parameters
maxlen = 30 # FIXME 
labels = 2

In [5]:
input = pd.read_csv("data_new/merged.csv",header=None)
input.columns = ['first', 'last','b_or_n']

# remove encode 
input['first'] = input['first'].str[2:-1]
input['last'] = input['last'].str[2:-1]

input['firstlen']= [len(str(i)) for i in input['first']]
input1 = input[(input['firstlen'] >= 2) ] #FIXME
print(input1)

                       first              last  b_or_n  firstlen
0             ithalohfonseca        chaverinho       1        14
1                    gustavo  gon\xc3\xa7alves       1         7
2                     rafael           geraldo       1         6
3                      anton          prasetyo       1         5
4                   cristyan            victor       1         8
5      valsemorborgesdesouza              neto       1        21
6            diegodereknobre             nobre       1        15
7                 paulonatan            santos       1        10
8              jardelurgalde          oliveira       1        13
9                   deolindo           barbosa       1         8
10         gabrielbitencourt         figueredo       1        17
11                   eduardo          carvalho       1         7
12                   matheus               rct       1         7
13                  fernando             cesar       1         8
14                   gust

In [5]:
firsts = input['first']

labels = input['b_or_n']
vocab = set(' '.join([str(i) for i in firsts]))
vocab.add('END')
len_vocab = len(vocab)

In [7]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [9]:
#train test split 
msk = np.random.rand(len(input1)) < 0.8

train = input1[msk]
test = input1[~msk]

print(train)

                   first              last  b_or_n  firstlen
1                gustavo  gon\xc3\xa7alves       1         7
2                 rafael           geraldo       1         6
3                  anton          prasetyo       1         5
7             paulonatan            santos       1        10
8          jardelurgalde          oliveira       1        13
10     gabrielbitencourt         figueredo       1        17
12               matheus               rct       1         7
13              fernando             cesar       1         8
14               gustavo           batalha       1         7
15               muhamad           riyandi       1         7
16      oohtalldomatheus               pjl       1        16
17               gabriel     ara\xc3\xbajo       1         7
19                thiago          urameshi       1         6
20             juliandra         bimantara       1         9
21               gustavo            araujo       1         7
22             guilherme

In [12]:
def set_flag(i):
    tmp = np.zeros(39);
    tmp[i] = 1
    return(tmp)

In [14]:
# Truncating and padding training data
train_X = []
train_Y = []
trunc_train_first = [str(i)[0:maxlen] for i in train['first']]
for i in trunc_train_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train['b_or_n']:
    if i == 1:
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [15]:
np.asarray(train_X).shape

(47840, 30, 39)

In [16]:
np.asarray(train_Y).shape

(47840, 2)

In [17]:
#build the model: 2 stacked LSTM
print('Building LSTM model')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))

# Softmax activation function
model.add(Activation('softmax'))

# Cross-entropy loss, metric is accuracy
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...


In [18]:
# Truncating and padding test data

test_X = []
test_Y = []
trunc_test_first = [str(i)[0:maxlen] for i in test['first']]
for i in trunc_test_first:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test['b_or_n']:
    if i == 1:
        test_Y.append([1,0])
    else:
        test_Y.append([0,1]) 

In [19]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(12156, 30, 39)
(12156, 2)


In [ ]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=10,validation_data=(test_X, test_Y))

/Users/DuaaTashkandi/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 47840 samples, validate on 12156 samples
Epoch 1/10
47840/47840 [==============================] - 813s 17ms/step - loss: 0.6112 - acc: 0.6554 - val_loss: 0.5452 - val_acc: 0.7470
Epoch 2/10
47000/47840 [============================>.] - ETA: 13s - loss: 0.5431 - acc: 0.7449

In [290]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,nb_epoch=50,validation_data=(test_X, test_Y))

Train on 12198 samples, validate on 3028 samples
Epoch 1/50
12198/12198 [==============================] - 145s - loss: 0.4107 - acc: 0.8137 - val_loss: 0.4408 - val_acc: 0.7966
Epoch 2/50
12198/12198 [==============================] - 144s - loss: 0.3912 - acc: 0.8254 - val_loss: 0.4479 - val_acc: 0.7936
Epoch 3/50
12198/12198 [==============================] - 145s - loss: 0.3927 - acc: 0.8228 - val_loss: 0.4511 - val_acc: 0.7982
Epoch 4/50
12198/12198 [==============================] - 145s - loss: 0.3730 - acc: 0.8344 - val_loss: 0.4253 - val_acc: 0.8071
Epoch 5/50
12198/12198 [==============================] - 145s - loss: 0.3640 - acc: 0.8396 - val_loss: 0.4240 - val_acc: 0.8164
Epoch 6/50
12198/12198 [==============================] - 145s - loss: 0.3490 - acc: 0.8505 - val_loss: 0.4183 - val_acc: 0.8180
Epoch 7/50
12198/12198 [==============================] - 145s - loss: 0.3411 - acc: 0.8542 - val_loss: 0.4089 - val_acc: 0.8243
Epoch 8/50
12198/12198 [========================

In [460]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3028/3028 [==============================] - 16s    
Test score: 0.448404541104
Test accuracy: 0.864266842879
